<a id="top"></a>
# Creating cf-compliant netCDF files in R

***

## Learning Objectives

At the end of this tutorial, learners will be able to:
- read binary data in R
- explain the netCDF data format and the "Climate and Forecast (CF)" meta data standard
- write netCDF data in R

As an example we will use precipitation data from a gauge-adjusted radardataset of Deutscher Wetterdienst (DWD).

## Introduction

The "radar climatology" of Deutscher Wetterdienst (DWD) contains precipitation measurements based on radardata which have been quality-controlled and adjusted. It offers continuous observations of precipitation in space and time with a spatial resolution of 1 km * 1km and a temporal resolution of 1 h (the so-called RW-product) or 5 min (YW-product). The access to the dataset and further information are available here:
DOI:_10.5676/DWD/RADKLIM_RW_V2017.002_

The data is available in a binary format which will be converted to the netCDF format in this tutorial. 


## Packages
The following r-packages are required:
- ncdf4 (handling netcdf data)
- chron (handling date and time values)
- fields (plotting data)

In [ ]:
#install.packages("ncdf4")
#install.packages("chron")
#install.packages("fields")

library(ncdf4)
library(chron)
library(fields)

## 1. Download the data

We start by downloading the data from the opendata-Server of Deutscher Wetterdienst. We arbitrarily choose the data for January 2001 as example data which corresponds to the file `RW2017.002_200101.tar.gz`.

In [ ]:
system("mkdir ./data")
system("wget https://opendata.dwd.de/climate_environment/CDC/grids_germany/hourly/radolan/reproc/2017_002/bin/2001/RW2017.002_200101.tar.gz", intern=TRUE)

## 2. Untar the data

The data is stored as monthly tar-archives. By untaring the data, files will be created which adhere to the following naming convention:
`raa01-rw2017.002_10000-************-dwd---bin`
where `*` serves as wildcard to denote the time and date of the data in the format `YYMMDDHHMM`. 

In [ ]:
system("tar -xvf RW2017.002_200101.tar.gz -C data", intern=TRUE)

## 3. Read the data

The data are stored in the so-called `RADKLIM-Komposit` format. In this format, every file consists of an ASCII part which contains meta data and a binary part which contains the actual precipitation data. The format is described in detail in the following document: 
https://www.dwd.de/DE/leistungen/radarklimatologie/radklim_kompositformat_1_0.pdf

### 3.1. Metadata

We extract the metadata `date` and `time` from the file names all files for January and save it in the `time vector` variable. 

In [ ]:
filelist_name_only<-dir("./data",pattern="raa01-rw2017")
filelist_full_path<-normalizePath(dir("./data",pattern="raa01-rw2017",full.names = T))
nfiles<-length(filelist_name_only)
time_vector<-vector(mode="double",length=nfiles)

for(i in 1:nfiles){
    year<-substr(as.character(filelist_name_only[i]),24,25)
    month<-substr(as.character(filelist_name_only[i]),26,27)
    day<-substr(as.character(filelist_name_only[i]),28,29)
    hour<-substr(as.character(filelist_name_only[i]),30,31)
    minute<-substr(as.character(filelist_name_only[i]),32,33)
    
    time_vector[i]=chron(dates(c(paste(month,"/",day,"/20",year,sep=""))),times=times(c(paste(hour,":",minute,":00"))),origin=c(01,01,1900))
}

### 3.2. Binary data

In [ ]:
radarmatrix<-array(data=0,dim=c(1100,900,nfiles))

#Loop over 24 files (equal to 1 day)

for(i in 1:nfiles){
    con=file(filelist_full_path[i],"rb")
    header1<-readChar(con,92)
    length_of_stations_in_header<-as.numeric(readChar(con,2))
    stations<-readChar(con,length_of_stations_in_header)
    ext<-readChar(con,1)
    x<-readBin(con,"integer",n=990000,size=2,signed=F,endian="little")
    close(con)
    temp<-matrix(x,ncol=(900),nrow=1100,byrow=T)
    temp[(which(temp==10692))]<-NA
    radarmatrix[,,i]<-temp
}

radarmatrix[which(radarmatrix>1000)]<-NA
radarmatrix<-radarmatrix/10;

## 4. Process data

In this section, the `radarmatrix` which we just stored will be saved as netCDF file.

### 4.1. Calculate the grid information

First of all, we need to calculate the grid information, i.e. the longitude and latitude values for our data. The data is stored in polar stereographic projection.

In [ ]:
x.vec <- seq(-442.962,456.038,1)    #length(900)
y.vec <- seq(-4758.145,-3659.145,1) #length(1100)
nx <- length(x.vec)      #x-direction of whole GER-Radardomain  #old domain: 900 new: 1100
ny <- length(y.vec)      #y-direction of whole GER-Radardomain  # old and new: 900

lon0=10
lat0=60 #Reference Point in Coordinates
EARTH_R=6370.04 #EARH RADIUS

lat=array(dim=c(nx,ny))
lon=array(dim=c(nx,ny))

xbounds=array(dim=c(nx,ny,4))
ybounds=array(dim=c(nx,ny,4))

x.v=c(-0.5,0.5,0.5,-0.5)
y.v=c(-0.5,-0.5,0.5,0.5)


for (i in 1:nx){
        for (j in 1:ny){
                lon[i,j]=(180/pi*atan(-(x.vec[i])/(y.vec[j]))+lon0)
                lat[i,j]=180/pi*asin((EARTH_R^2*(1+sin(pi/180*lat0))^2-(x.vec[i]^2+y.vec[j]^2))/(EARTH_R^2*(1+sin(pi/180*lat0))^2+(x.vec[i]^2+y.vec[j]^2)))
                for (k in 1:4){
                    xbounds[i,j,k]=(180/pi*atan(-(x.vec[i]+x.v[k])/(y.vec[j]+y.v[k]))+lon0)
                    ybounds[i,j,k]=180/pi*asin((EARTH_R^2*(1+sin(pi/180*lat0))^2-((x.vec[i]+x.v[k])^2+y.vec[j]^2))/(EARTH_R^2*(1+sin(pi/180*lat0))^2+((x.vec[i]+x.v[k])^2+(y.vec[j]+y.v[k])^2)))
                }
        }
}

### 4.2. Writing the netCDF-file

Writing the netcdf file consists of the following steps:
1. Defining the dimensions of the netcdf-file (function call: ncdim_def)
2. Defining the variables, both coordinate and data variables (function call: ncvar_def)
3. Creating the netcdf-File (function call: nc_create)
4. Inserting the variables into the netCDF-file
5. Adding global and variable-specific metadata attributes 

In [ ]:
# 1. Defining the dimensions 
x<-ncdim_def("x",create_dimvar=F,"",1:900)
y<-ncdim_def("y",create_dimvar=F,"",1:1100)
timedim<-ncdim_def("time","days since 1900-01-01 00:00:00.0 -0:00",time_vector)

fillvalue<-1e32
dlname <- "precipitation_amount"

# 2. Defining the variables
prec_def <- ncvar_def("prec","kg m-2",list(x,y,timedim),fillvalue,dlname,prec="single")
lon_def<-ncvar_def(name="longitude",units="degrees_east",dim=list(x,y))
lat_def<-ncvar_def(name="latitude",units="degrees_north",dim=list(x,y))

# 3. Creating the netCDF-file
ncfname <- paste("/data/icdc_main/PROJECTS/nfdi4earth/TA1_3/workshop_netcdf/netcdf-workshop-jupyterbook/prec_rw_",year,month,".nc",sep="")
ncout <- nc_create(ncfname,vars=list(prec_def,lon_def,lat_def),force_v4=T)

# 4. Inserting the variables
ncvar_put(ncout,lon_def,lon)
ncvar_put(ncout,lat_def,lat)
ncvar_put(ncout,prec_def,aperm(radarmatrix,c(2,1,3)))

# 5. Adding attributes
#  5.1. Variable attributes
ncatt_put(ncout,"longitude","axis","X") #,verbose=FALSE) #,definemode=FALSE)
ncatt_put(ncout,"latitude","axis","Y")
ncatt_put(ncout,"time","axis","T")
ncatt_put(ncout,varid="prec",attname="coordinates",attval="longitude latitude")
ncatt_put(ncout,varid="longitude",attname="standard_name",attval="longitude")
ncatt_put(ncout,varid="latitude",attname="standard_name",attval="latitude")
#  5.2. Global attributes
ncatt_put(ncout,0,"title","Radar-based Precipitation Climatology for Germany")
ncatt_put(ncout,0,"institution","Deutscher Wetterdienst (DWD)")
ncatt_put(ncout,0,"source","Radar composit, Gauge-adjusted")
ncatt_put(ncout,0,"references","http://dx.doi.org/10.5676/DWD/RADKLIM_RW_V2017.002")
ncatt_put(ncout,0,"history","transformation to netcdf with procedure by Christopher Purr (University of Hamburg) 2023/01/01")
ncatt_put(ncout,0,"Conventions","CF-1.8")

# Close the file and write data to disk
nc_close(ncout)

In [ ]:
image.plot(seq(1:900),seq(1:1100),t(radarmatrix[,,20]),asp=1)

## 5. Checking if the netcdf-file conforms to the cf-Standard

To check whether the netCDF-file we created conforms to the CF-Standard, you can use the "CF-Checker" at 
https://compliance.ioos.us/index.html

Just choose CF-(1.6 to 1.8) and submit the created nc-file.

## Further material

- https://unidata.github.io/python-training/workshop/CF%20Conventions/netcdf-and-cf-the-basics/
- CF-Conventions - Overview: https://cfconventions.org/
- CF-Conventions - current version: http://cfconventions.org/cf-conventions/cf-conventions.html